In [90]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [91]:
path = '/content/drive/MyDrive/Colab Notebooks/DACON CUP'

user =  pd.read_csv(path + '/open_data/info_user.csv', encoding = 'euc-kr')
login = pd.read_csv(path +'/open_data/info_login.csv', encoding = 'euc-kr')
competition = pd.read_csv(path +'/open_data/info_competition.csv', encoding = 'euc-kr')
info_submission =  pd.read_csv(path +'/open_data/info_submission.csv', encoding = 'euc-kr')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [308]:
train = pd.read_csv(path + '/open_data/train.csv', encoding = 'euc-kr')
train.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2018-09-09 00:00:00,19,19,8,206
1,2018-09-09 01:00:00,20,19,9,259
2,2018-09-09 02:00:00,12,9,1,48
3,2018-09-09 03:00:00,10,10,2,102
4,2018-09-09 04:00:00,6,5,3,18


In [309]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19008 entries, 0 to 19007
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  19008 non-null  object
 1   사용자       19008 non-null  int64 
 2   세션        19008 non-null  int64 
 3   신규방문자     19008 non-null  int64 
 4   페이지뷰      19008 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 742.6+ KB


In [310]:
# date 기준으로.

train['DateTime'] = pd.to_datetime(train.DateTime) #dtype: datetime으로 변경
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()
train.head()

,date,사용자,세션,신규방문자,페이지뷰
0,2018-09-09,281,266,73,1826
1,2018-09-10,264,247,51,2092
2,2018-09-11,329,310,58,1998
3,2018-09-12,300,287,45,2595
4,2018-09-13,378,344,50,3845


In [311]:
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 61
output_window = 61 # 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values


print('window_x.shape: ', window_x.shape)
print('window_y.shape: ', window_y.shape)

window_x.shape:  (670, 61, 4)
window_y.shape:  (670, 61, 4)


In [300]:
# window_x

- 시계열 데이터 임베딩 찾아서 적용해보기
- 스케일링 다르게 해보기
- dropout 적용해보기

In [301]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model


optimizer = Adam, loss = mse, activation_function = 따로지정x

In [322]:
model = Sequential()
model.add(LSTM(32, input_shape=(61, 4), return_sequences=True))  # ()
model.add(LSTM(32, return_sequences=True)) 
model.add(LSTM(32, return_sequences=True)) 
model.add(LSTM(32, return_sequences=True)) 
model.add(LSTM(32, return_sequences=True)) 
model.add(TimeDistributed(Dense(4)))

model.summary()

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt)

model.fit(window_x, window_y, epochs=100, batch_size=50)

# lstm2층 - epochs100, batch_size=10

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 61, 32)            4736      
_________________________________________________________________
lstm_26 (LSTM)               (None, 61, 32)            8320      
_________________________________________________________________
lstm_27 (LSTM)               (None, 61, 32)            8320      
_________________________________________________________________
lstm_28 (LSTM)               (None, 61, 32)            8320      
_________________________________________________________________
lstm_29 (LSTM)               (None, 61, 32)            8320      
_________________________________________________________________
time_distributed_10 (TimeDis (None, 61, 4)             132       
Total params: 38,148
Trainable params: 38,148
Non-trainable params: 0
_________________________________________________

## test : last_month

In [323]:
# train은 이미 위에서 minmax scaler적용되어있음.

last_month = train.iloc[-61:,1:].values[np.newaxis,...] # np.newaxis: 3D로 차원을 늘려줌. 
last_month.shape

(1, 61, 4)

In [324]:
pred_month = model.predict(last_month)
pred_month[:5]

array([[[0.43963647, 0.43365198, 0.32959282, 0.4240301 ],
        [0.5167538 , 0.50954247, 0.3996908 , 0.49516553],
        [0.5285641 , 0.5220842 , 0.42350447, 0.48978907],
        [0.4824617 , 0.4778763 , 0.414187  , 0.4362203 ],
        [0.48066807, 0.48141915, 0.4268977 , 0.42275542],
        [0.3485274 , 0.34517598, 0.28855813, 0.28458583],
        [0.35799232, 0.35710925, 0.28475055, 0.32664666],
        [0.6784514 , 0.6827958 , 0.6185802 , 0.65212035],
        [0.6934716 , 0.69402695, 0.60501873, 0.6752107 ],
        [0.5951985 , 0.60278666, 0.49803567, 0.5573926 ],
        [0.5846635 , 0.60112315, 0.533925  , 0.5143457 ],
        [0.85704595, 0.90893114, 0.9535506 , 0.8849674 ],
        [0.40649867, 0.40829855, 0.38670433, 0.3048669 ],
        [0.2778231 , 0.26801777, 0.21661475, 0.22114849],
        [0.45005727, 0.44205242, 0.31538743, 0.5300254 ],
        [0.55307066, 0.53001297, 0.42574823, 0.6445355 ],
        [0.55094147, 0.5259095 , 0.4103186 , 0.6477212 ],
        [0.484

In [325]:
pred_month = pred_month.reshape(output_window,4)
pred_month = pred_month * size.values + mini.values
pred_month = pred_month.astype(int)
pred_month


array([[  2382,   2342,    558,  65910],
       [  2800,   2752,    676,  76967],
       [  2864,   2819,    716,  76131],
       [  2614,   2581,    700,  67805],
       [  2604,   2600,    722,  65712],
       [  1888,   1864,    488,  44236],
       [  1940,   1929,    482,  50773],
       [  3676,   3687,   1046, 101363],
       [  3757,   3748,   1023, 104952],
       [  3225,   3255,    842,  86639],
       [  3168,   3246,    903,  79948],
       [  4643,   4908,   1612, 137555],
       [  2203,   2205,    654,  47388],
       [  1505,   1448,    367,  34375],
       [  2438,   2387,    534,  82385],
       [  2996,   2862,    720, 100184],
       [  2985,   2840,    694, 100679],
       [  2623,   2519,    601,  80420],
       [  2487,   2415,    548,  71974],
       [  1749,   1724,    393,  46535],
       [  1738,   1707,    394,  47427],
       [  3167,   3099,    786,  87075],
       [  3251,   3198,    900,  76100],
       [  2304,   2241,    557,  50257],
       [  2711, 

In [326]:
pred_month.shape

(61, 4)

## submit

In [327]:
submission =  pd.read_csv(path + '/open_data/submission.csv', encoding = 'euc-kr')
submission.shape

(61, 5)

In [328]:
pred_month.shape

(61, 4)

In [329]:
pred_df = pd.DataFrame(pred_month)
pred_df.head()

,0,1,2,3
0,2382,2342,558,65910
1,2800,2752,676,76967
2,2864,2819,716,76131
3,2614,2581,700,67805
4,2604,2600,722,65712


In [330]:
# 30까지 모두 채우면 다시 0부터.

for i in range(len(submission)):
  submission['사용자'][i] = pred_df[0][i]
  submission['세션'][i] = pred_df[1][i]
  submission['신규방문자'][i] = pred_df[2][i]
  submission['페이지뷰'][i] = pred_df[3][i]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: Se

In [331]:
submission.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,2382,2342,558,65910
1,2020-11-10,2800,2752,676,76967
2,2020-11-11,2864,2819,716,76131
3,2020-11-12,2614,2581,700,67805
4,2020-11-13,2604,2600,722,65712


In [336]:
submission.to_csv('61 lstm3.csv', index = False, encoding = 'euc-kr')

## 참조

In [ ]:
# 각 변수의 크기가 다르기 때문에 가중치를 부여한 RMSE로 모델의 성능을 평가

def dacon_rmse(true, pred):
    # true.shape // (N,4)
    # pred.shape // (N,4)
    # w0, w1, w2, w3 <= train.csv의 사용자 수, 세션 수, 신규 방문자 수, 페이지 뷰 수 4가지 항목별 평균값
    score = np.sqrt(np.mean(np.square(true[:,0] - pred[:,0]))) / w0 +\
            + np.sqrt(np.mean(np.square(true[:,1] - pred[:,1]))) / w1 +\
            + np.sqrt(np.mean(np.square(true[:,2] - pred[:,2]))) / w2 +\
            + np.sqrt(np.mean(np.square(true[:,3] - pred[:,3]))) / w3 +\
     return score

In [ ]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

train_sc

In [ ]:
# date 기준 변수 만들기

from datetime import datetime

def create_features(df):
    """
    Creates time series features from datetime index
    """
    # df['DateTime'] = df.index
    # df['hour'] = df['date'].dt.hour
    df['dayofweek'] = pd.to_datetime(df['date']).dt.dayofweek
    df['quarter'] = pd.to_datetime(df['date']).dt.quarter
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['year'] = pd.to_datetime(df['date']).dt.year
    df['dayofyear'] = pd.to_datetime(df['date']).dt.dayofyear
    df['dayofmonth'] = pd.to_datetime(df['date']).dt.day
    df['weekofyear'] = pd.to_datetime(df['date']).dt.weekofyear
    
    X = df[['사용자','세션','신규방문자','페이지뷰','date','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    X.set_index('date', inplace = True)
    X.index = pd.to_datetime(X.index)
    return X

## 예측값을 x로 다시 predict  
- 마이너스 값이 나와서 실패..

In [240]:
tr_month1 = pd.DataFrame(pred_month)

In [241]:
# 위 예측값을 x로 사용하기.
mini1 = tr_month1.iloc[:,1:].min()
size1 = tr_month1.iloc[:,1:].max() - tr_month1.iloc[:,1:].min()
tr_month1.iloc[:,1:] = (tr_month1.iloc[:,1:] -  mini1) / size1

tr_month1 = tr_month1.iloc[:,:].values[np.newaxis,...] # np.newaxis: 3D로 차원을 늘려줌. 
tr_month1.shape

(1, 30, 4)

In [242]:
pred_month1 = model.predict(tr_month1)
pred_month1[:5]

array([[[ 0.18093495,  0.15617856,  0.17545578,  0.07859027],
        [ 0.09990777,  0.06255519,  0.09158635, -0.03202646],
        [ 0.1238751 ,  0.08535202,  0.10137758, -0.0130481 ],
        [ 0.1609668 ,  0.12201263,  0.13407502,  0.02669705],
        [ 0.17643648,  0.13653305,  0.14765052,  0.04409892],
        [ 0.17911391,  0.13828306,  0.1470722 ,  0.04774056],
        [ 0.18096438,  0.13957691,  0.14543536,  0.05040792],
        [ 0.18431361,  0.14262158,  0.14594844,  0.05457786],
        [ 0.18804072,  0.14617258,  0.14746507,  0.05902395],
        [ 0.19144723,  0.14947562,  0.14904726,  0.06302298],
        [ 0.19444391,  0.15241605,  0.15047295,  0.06651456],
        [ 0.19708903,  0.15503576,  0.15175058,  0.0695814 ],
        [ 0.19941911,  0.15735956,  0.1528904 ,  0.07227365],
        [ 0.20145619,  0.1594011 ,  0.15389048,  0.07462133],
        [ 0.20322528,  0.16118066,  0.15475564,  0.07665686],
        [ 0.20475666,  0.1627254 ,  0.15549937,  0.07841668],
        

In [243]:
pred_month1 = pred_month1.reshape(output_window,4)
pred_month1 = pred_month1 * size.values + mini.values
pred_month1 = pred_month1.astype(int)
pred_month1


array([[  981,   844,   297, 12217],
       [  542,   338,   155, -4975],
       [  672,   461,   172, -2026],
       [  872,   659,   227,  4151],
       [  956,   738,   250,  6856],
       [  971,   747,   249,  7422],
       [  981,   754,   246,  7837],
       [  999,   771,   247,  8485],
       [ 1019,   790,   250,  9176],
       [ 1038,   808,   252,  9797],
       [ 1054,   823,   255, 10340],
       [ 1068,   838,   257, 10817],
       [ 1081,   850,   259, 11235],
       [ 1092,   861,   261, 11600],
       [ 1101,   871,   262, 11917],
       [ 1110,   879,   263, 12190],
       [ 1117,   886,   264, 12426],
       [ 1123,   893,   265, 12630],
       [ 1128,   898,   266, 12805],
       [ 1133,   903,   267, 12957],
       [ 1137,   907,   267, 13087],
       [ 1140,   910,   268, 13200],
       [ 1143,   913,   268, 13297],
       [ 1146,   916,   269, 13382],
       [ 1148,   918,   269, 13455],
       [ 1150,   920,   269, 13518],
       [ 1152,   922,   269, 13573],
 

#### 윈도우 크기 다르게 해서 train하고 test해보기

aa

## test = train[-61]

In [201]:
test = train[-61:]
test

,date,사용자,세션,신규방문자,페이지뷰
731,2020-09-09,0.497877,0.490461,0.355030,0.377532
732,2020-09-10,0.452464,0.450824,0.324260,0.339207
733,2020-09-11,0.405206,0.397111,0.278107,0.321836
734,2020-09-12,0.298320,0.295425,0.194083,0.242021
735,2020-09-13,0.342810,0.333951,0.198817,0.288169
...,...,...,...,...,...
787,2020-11-04,0.833487,0.828116,0.707101,0.724949
788,2020-11-05,0.766845,0.747546,0.617160,0.662015
789,2020-11-06,0.676020,0.662160,0.487574,0.566244
790,2020-11-07,0.456157,0.447490,0.313609,0.369188


In [202]:
# scaling

input_window = 30
output_window = 30

test_x = np.zeros((test.shape[0] - (input_window + output_window), input_window, 4))
test_y = np.zeros((test.shape[0] - (input_window + output_window), output_window, 4))

for start in range(test.shape[0] - (input_window + output_window)):
    end = start + input_window    
    test_x[start,:, :] = test.iloc[start : end                , 1: ].values
    test_y[start,:, :] = test.iloc[end   : end + output_window, 1: ].values


print('test_x.shape: ', test_x.shape)
print('test_y.shape: ', test_y.shape)

test_x.shape:  (1, 30, 4)
test_y.shape:  (1, 30, 4)


In [205]:
pred = model.predict(test_x)
pred[:5]

array([[[0.40022817, 0.39372706, 0.27692318, 0.33888835],
        [0.38961065, 0.3810348 , 0.25960314, 0.3113099 ],
        [0.4294502 , 0.41507578, 0.28880233, 0.3883028 ],
        [0.57350516, 0.56259114, 0.4501635 , 0.5218927 ],
        [0.4765348 , 0.467364  , 0.3573951 , 0.40904856],
        [0.4156783 , 0.4077924 , 0.30611813, 0.35120863],
        [0.39636064, 0.39218706, 0.30435696, 0.30528963],
        [0.36307785, 0.3584634 , 0.27156314, 0.28400347],
        [0.27903116, 0.27569184, 0.22178051, 0.19545761],
        [0.37462008, 0.3721298 , 0.3127888 , 0.26252687],
        [0.47159705, 0.46326333, 0.39487627, 0.3773024 ],
        [0.5758832 , 0.5726595 , 0.5498195 , 0.51826733],
        [0.4800876 , 0.47606337, 0.40594587, 0.41935587],
        [0.5029881 , 0.49729228, 0.44717255, 0.40659106],
        [0.41189307, 0.40996557, 0.32963312, 0.3260718 ],
        [0.32444862, 0.31973892, 0.24362595, 0.24606156],
        [0.31358963, 0.30821908, 0.22619447, 0.24227071],
        [0.573

In [206]:
pred = pred.reshape(output_window,4)
pred = pred * size.values + mini.values
pred = pred.astype(int)
pred

array([[  2169,   2126,    469,  52676],
       [  2111,   2058,    439,  48389],
       [  2327,   2241,    489,  60357],
       [  3107,   3038,    761,  81121],
       [  2582,   2524,    604,  63581],
       [  2252,   2202,    518,  54591],
       [  2148,   2118,    515,  47454],
       [  1967,   1936,    459,  44145],
       [  1512,   1489,    375,  30382],
       [  2030,   2010,    529,  40807],
       [  2555,   2502,    668,  58647],
       [  3120,   3092,    930,  80557],
       [  2601,   2571,    687,  65183],
       [  2725,   2685,    756,  63199],
       [  2232,   2214,    558,  50684],
       [  1758,   1727,    412,  38248],
       [  1699,   1665,    383,  37658],
       [  3105,   3034,    777,  77979],
       [  3238,   3187,    846,  80160],
       [  3406,   3323,    811,  87295],
       [  3934,   3844,    924, 103179],
       [  3397,   3309,    775,  89027],
       [  2084,   2102,    496,  55332],
       [  2265,   2250,    480,  51135],
       [  5027, 

In [207]:
pred.shape

(30, 4)

In [ ]:
baseline = pd.read_csv(path + '/open_data/baseline.csv', encoding = 'euc-kr' )
baseline.head()

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,3252,3263,777,88002
1,2020-11-10,2825,2758,671,73749
2,2020-11-11,2614,2588,599,67300
3,2020-11-12,2391,2373,557,63723
4,2020-11-13,2063,2047,467,54340


In [ ]:
last_month = train.iloc[-30:,1:].values[np.newaxis,...].astype(float) #float() # <- 수정 후
last_month

In [ ]:

#last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device) <- 수정 전
last_month = train.iloc[-30:,1:].values[np.newaxis,...] # <- 수정 후
# last_month = torch.tensor(last_month).float() # <- 수정 후

for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model.predict(last_month)
    #last_month = torch.cat([last_month[-7:], next_week], axis = 1) <- 수정 전
    last_month = torch.cat([last_month[:,7:,:], next_week], axis = 1)# <- 수정 후

    pred_week = next_week.reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission